In [1]:
from lst_generator import LSTGenerator

Using TensorFlow backend.


In [1]:
import utils, keras, numpy as np, pandas as pd

Using TensorFlow backend.
/home/pietro/miniconda3/envs/cta-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pietro/miniconda3/envs/cta-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pietro/miniconda3/envs/cta-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pietro/m

In [2]:
# file = ['/mnt/Dati/CTA_DATA_EXT/data/interp/proton/proton_20deg_180deg_runs1001-1010___cta-prod3-demo-2147m-LaPalma-baseline-mono_interp.h5']
file = ['/mnt/Dati/CTA_DATA_EXT/data/interp/gamma-diffuse_20deg_180deg_runs131-140___cta-prod3-demo-2147m-LaPalma-baseline-mono_interp.h5']

In [4]:
gen = LSTGenerator( 
    h5files=file, 
    feature='direction',
    shuffle=False,
    intensity=1000,
    leakage2_intensity=0.2,
    arrival_time=True
    )

In [6]:
info = gen.get_all_info()
info

,event_index,file_index,image_index,energy_true,alt,az,d_alt_true,d_az_true,intensity,leakage2,class
0,4,0,6,0.974951,73.103597,179.034708,3.105821,-0.280955,1351.031990,0.005802,1.0
1,5,0,9,0.974951,73.103597,179.034708,3.105821,-0.280955,1031.697512,0.011965,1.0
2,6,0,14,0.974951,73.103597,179.034708,3.105821,-0.280955,1456.224572,0.017741,1.0
3,33,0,60,4.605054,66.984136,174.275323,-2.910736,-2.238002,1105.342490,0.004631,1.0
4,33,0,61,4.605054,66.984136,174.275323,-2.910736,-2.238002,2406.207073,0.023401,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1659,11964,0,19659,0.182198,70.118626,172.859124,0.260646,-2.422890,1405.290170,0.014046,1.0
1660,11980,0,19681,0.167076,69.788674,174.354840,-0.121114,-1.947536,1423.621027,0.000000,1.0
1661,11980,0,19682,0.167076,69.788674,174.354840,-0.121114,-1.947536,1152.183150,0.000000,1.0
1662,12018,0,19748,0.358167,68.268355,185.157631,-1.650896,1.908230,4261.646750,0.135249,1.0


In [7]:
df = info[['alt','az']]
df

,alt,az
0,73.103597,179.034708
1,73.103597,179.034708
2,73.103597,179.034708
3,66.984136,174.275323
4,66.984136,174.275323
...,...,...
1659,70.118626,172.859124
1660,69.788674,174.354840
1661,69.788674,174.354840
1662,68.268355,185.157631


In [8]:
df[['d_alt','d_az']] = gen.get_AltAz()
df

,alt,az,d_alt,d_az
0,73.103597,179.034708,3.105821,-0.280955
1,73.103597,179.034708,3.105821,-0.280955
2,73.103597,179.034708,3.105821,-0.280955
3,66.984136,174.275323,-2.910736,-2.238002
4,66.984136,174.275323,-2.910736,-2.238002
...,...,...,...,...
1659,70.118626,172.859124,0.260646,-2.422890
1660,69.788674,174.354840,-0.121114,-1.947536
1661,69.788674,174.354840,-0.121114,-1.947536
1662,68.268355,185.157631,-1.650896,1.908230


model = '/home/pietro/CTA/ctrained/selected/VGG16_2020-04-21_11-01/VGG16_13_0.88753_0.86502.h5'

#mdl = keras.models.load_model(model)

pr_labels = mdl.predict_generator(generator=gen, steps=None, max_queue_size=10, workers=4, use_multiprocessing=False, verbose=1)

info = info[:len(pr_labels)]
info['gammaness'] = pr_labels

labels = gen.evaluate_gammaness(model)

In [ ]:
print('Loading model...')
altaz_model = '/home/pietro/CTA/dtrained/VGG16_direction_2020-05-21_14-40/models/VGG16_11_0.04807_0.06395.h5'
mdl = keras.models.load_model(altaz_model)

print('Predicting direction...')
predictions = mdl.predict_generator(
    generator=gen, 
    max_queue_size=10, 
    workers=4, 
    use_multiprocessing=False, 
    verbose=1
)

In [ ]:
info['delta_alt_reco'] = predictions[:, 0]
info['delta_az_reco'] = predictions[:, 1]

theta_square = np.power(( info['delta_alt'] - info['delta_alt_reco'] ), 2) + np.power(( info['delta_az'] - info['delta_az_reco'] ), 2)


In [ ]:
info['theta_square'] = theta_square

In [ ]:
above1 = info[info['theta_square']>1].reset_index(drop=True)

In [ ]:
print('# of theta2 >1: {:.2f}%'.format(len(above1)/len(info)*100))

In [ ]:
around1 = info[(info['theta_square']>0.9) & (info['theta_square']<1.1)].reset_index(drop=True)

In [ ]:
print('# of 0.9 < theta2 < 1.1: {:.2f}%'.format(len(around1)/len(info)*100))

In [ ]:
x, _ = gen.get_charges(around1)

In [ ]:
geom = CameraGeometry.from_name('LSTCam')

# GENERATOR 2

In [1]:
import utils, keras, numpy as np, pandas as pd
import lst_generator_2 as lg2

Using TensorFlow backend.
/home/pietro/miniconda3/envs/cta-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pietro/miniconda3/envs/cta-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pietro/miniconda3/envs/cta-dev/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pietro/m

In [2]:
# file = ['/mnt/Dati/CTA_DATA_EXT/data/interp/proton/proton_20deg_180deg_runs1001-1010___cta-prod3-demo-2147m-LaPalma-baseline-mono_interp.h5']
file = ['/mnt/Dati/CTA_DATA_EXT/data/interp/gamma-diffuse_20deg_180deg_runs131-140___cta-prod3-demo-2147m-LaPalma-baseline-mono_interp.h5']

In [3]:
gen2 = lg2.LSTGenerator( 
    h5files=file, 
    feature='direction',
    shuffle=False,
    intensity=1000,
    leakage2_intensity=0.2,
    arrival_time=True
    )

In [4]:
gen2.get_all_info()

,file_idx,img_idx,event_idx,intensity,leakage2,class,energy_true,az,alt,d_alt,d_az
0,0.0,60.0,33.0,1105.342490,0.004631,1.0,0.663235,66.984136,174.275323,-2.910736,-2.238002
1,0.0,61.0,33.0,2406.207073,0.023401,1.0,0.663235,66.984136,174.275323,-2.910736,-2.238002
2,0.0,74.0,40.0,2498.235302,0.000000,1.0,1.344816,65.956186,186.122368,-3.918394,2.496347
3,0.0,75.0,40.0,1884.369874,0.000000,1.0,1.344816,65.956186,186.122368,-3.918394,2.496347
4,0.0,76.0,40.0,3713.471740,0.000000,1.0,1.344816,65.956186,186.122368,-3.918394,2.496347
...,...,...,...,...,...,...,...,...,...,...,...
891,0.0,19135.0,11658.0,1307.558145,0.144394,1.0,0.282922,70.226065,186.874467,0.357017,2.320804
892,0.0,19138.0,11659.0,8236.437465,0.007149,1.0,0.282922,70.226065,186.874467,0.357017,2.320804
893,0.0,19189.0,11691.0,1302.902082,0.140099,1.0,0.751374,64.353831,177.541000,-5.624607,-1.069168
894,0.0,19255.0,11724.0,2818.215144,0.142535,1.0,0.494754,70.039724,168.884309,0.384527,-3.773601


In [17]:
gen2.get_AltAz(nom_frame=False)

array([[ 66.98413647, 174.27532294],
       [ 66.98413647, 174.27532294],
       [ 65.95618617, 186.12236801],
       ...,
       [ 64.35383062, 177.5410003 ],
       [ 70.03972405, 168.88430927],
       [ 70.03972405, 168.88430927]])